In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
import time

# 데이터 불러오기

In [14]:
df = pd.read_csv("data/분양권/분양권_2020이전.csv")
df.shape

C:\Users\medici\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,6,7,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(216037, 12)

In [15]:
# 시군구 단지명만 리스트 가져오기
sidan = df[['시군구', '단지명']]
sidan = sidan.drop_duplicates()
sidan = pd.DataFrame(sidan)
sidan.head()

,시군구,단지명
0,서울특별시 강남구 개포동,디에이치 아너힐즈
11,서울특별시 강남구 개포동,래미안 블레스티지
116,서울특별시 강남구 대치동,대치 SK VIEW
134,서울특별시 강남구 대치동,래미안대치청실
136,서울특별시 강남구 대치동,현대썬앤빌 테헤란


In [16]:
# 매매 시군구_단지명 불러오기
mae_sidan = pd.read_csv('data/etc/시군구_단지명.csv')
mae_sidan.head()

,시군구,단지명
0,경기도 고양덕양구 고양동,고양동우방유쉘아파트
1,경기도 고양덕양구 고양동,삼성
2,경기도 고양덕양구 고양동,윤창
3,경기도 고양덕양구 고양동,청구
4,경기도 고양덕양구 고양동,푸른마을 10단지


In [17]:
# 매매 시군구_단지명 테이블을 리스트로 변경
mae_sidan_list = mae_sidan.values.tolist()
# 제외리스트
dropped = []

for si, dan in mae_sidan_list:
    check = sidan[(sidan.시군구 == si) & (sidan.단지명 == dan)]
    # 데이터가 하나라도 있다면
    if check.shape[0] > 0:
        # 그 시군구, 단지명 인덱스를 저장
        idx = check.index.tolist()
        # 제외리스트에 담기
        dropped.extend(idx)
        
# 전체 성현의 몫 인덱스에서 제외리스트를 뺀다면 그 나머지만 하면 됨.
leftover = set(sidan.index.tolist()) - set(dropped)
# sh_new = sh.loc[leftover]
sidan_new = sidan.loc[leftover]
# sh_new
sidan_new.head()

,시군구,단지명
0,서울특별시 강남구 개포동,디에이치 아너힐즈
8196,서울특별시 송파구 장지동,위례중앙푸르지오1단지 (BL5)
59398,경기도 화성시 장지동,동탄 자이파밀리에
163847,경기도 성남분당구 대장동,판교 퍼스트힐 푸르지오 A2BL
163848,경기도 성남분당구 대장동,판교대장A6 힐스테이트 판교 엘포레


In [18]:
df.replace({'단지명':'디에이치 아너힐즈'},{'단지명':'디에이치아너힐즈'}, inplace=True)
df.replace({'단지명':'래미안 블레스티지'},{'단지명':'래미안블레스티지'}, inplace=True)
df.replace({'단지명':'도곡 한라비발디'},{'단지명':'한라비발디'}, inplace=True)
df.replace({'단지명':'삼성동 센트럴 IPARK '},{'단지명':'삼성동센트럴아이파크'}, inplace=True)
df.replace({'단지명':'강남더샵포레스트'},{'단지명':'강남 더샵 포레스트'}, inplace=True)
df.replace({'단지명':'삼성동 센트럴 IPARK '},{'단지명':'삼성동센트럴아이파크'}, inplace=True)
df.replace({'단지명':'래미안 루체하임'},{'단지명':'래미안 개포 루체하임'}, inplace=True)
df.replace({'단지명':'청담 린든그로브'},{'단지명':'청담린든그로브'}, inplace=True)
df.replace({'단지명':'래미안 루체하임'},{'단지명':'래미안 개포 루체하임'}, inplace=True)
df.replace({'단지명':'고덕 그라시움'},{'단지명':'고덕그라시움'}, inplace=True)
df.replace({'단지명':'고덕숲 I PARK '},{'단지명':'고덕숲아이파크'}, inplace=True)
df.replace({'단지명':'성내 올림픽파크 한양수자인'},{'단지명':'올림픽파크한양수자인'}, inplace=True)
df.replace({'단지명':'꿈의숲롯데캐슬(미아4구역)'},{'단지명':'꿈의숲롯데캐슬'}, inplace=True)
df.replace({'단지명':'수유 프리미어엠 주상복합 도시형생활주택'},{'단지명':'수유프리미어엠'}, inplace=True)
df.replace({'단지명':'고덕숲 I PARK '},{'단지명':'고덕숲아이파크'}, inplace=True)
df.replace({'단지명':'마곡힐스테이트마스터(13단지)'},{'단지명':'마곡13단지힐스테이트마스터'}, inplace=True)
df.replace({'단지명':'e편한세상 염창 (서울)'},{'단지명':'e편한세상 염창'}, inplace=True)
df.replace({'단지명':'e편한세상 서울대입구2차'},{'단지명':'e편한세상 서울대입구 2단지'}, inplace=True)

# 분양권 이름 변경

In [19]:
# 분양권 이름 변경 함수
def changeWord(before, after):
    
    condition = sidan_new[sidan_new.단지명 == before]
    
    # 같은 이름의 단지가 있다면 시군구를 추가로 입력하여 해당 단지만 이름을 바꾸기
    if condition.shape[0] >1:
        print('개수가 1개 이상입니다.')
        display(condition)
        sgg = input('시군구를 입력해주세요')
        
        df.loc[(df.시군구==sgg) & (df.단지명 == before), '단지명'] = after
        sidan_new.loc[(sidan_new.시군구==sgg) & (sidan_new.단지명==before), '단지명'] = after
        
        시군구단지명 = sidan_new[['시군구', '단지명']]
        시군구단지명 = 시군구단지명.drop_duplicates()
        시군구단지명 = 시군구단지명.sort_values(['시군구', '단지명'])
        display(시군구단지명[시군구단지명.단지명.isin(excepted) == False].shape)
        
    else:
        # 단지명 대체
        df.replace({'단지명': before},{'단지명': after}, inplace=True)
        sidan_new.replace({'단지명': before},{'단지명': after}, inplace=True)
        # 대체한 단지명 목록 저장
        change_dir[before] = after


        # 대체 후 시군구, 단지명
        시군구단지명 = sidan_new[['시군구', '단지명']]
        시군구단지명 = 시군구단지명.drop_duplicates()
        시군구단지명 = 시군구단지명.sort_values(['시군구', '단지명'])

        excepted.append(after)
        
        # 변경한 단지를 제외하고 남은 테이블을 출력하여 이어서 제거작업을 할 수 있게!
        display(시군구단지명[시군구단지명.단지명.isin(excepted) == False].shape)
    return

In [20]:
change_dir = {} # 바뀐 단지명 목록
excepted = []   # 바뀐 단지명 리스트

In [22]:
# 경기도
changeWord("래미안 구의 파크스위트", "래미안파크스위트")
changeWord(" 롯데캐슬 골드파크1차", "금천롯데캐슬 골드파크1차")
changeWord("태릉입구역 미라쉘", "미라쉘")
changeWord("방학지음재힐스", "방학 지음재힐스")
changeWord("보광타워 답십리(도시형)", "보광타워")
changeWord('장안 태영 데시앙', '동대문더퍼스트데시앙')
changeWord('휘경 해모로 프레스티지', '휘경 해모로 프레스티지')
changeWord('마포 웨스트리버 태영 데시앙', '마포웨스트리버태영데시앙')
changeWord('방배 마에스트로', '방배마에스트로')
changeWord('고양원흥A5BL 호반베르디움', '원흥호반베르디움더퍼스트5단지')
changeWord('고양 삼송 A10블록 동일스위트2차','삼송10단지동일스위트2차')
changeWord('삼송3차 아이파크','삼송3차아이파크')
changeWord('지축역 반도유보라','지축역북한산유보라')
changeWord('지축역 한림 풀에버', '지축역 한림풀에버')
changeWord('고양 향동지구 B-2블록 호반베르디움', 'DMC호반베르디움더포레2단지')
changeWord('고양 향동지구 B-3블록 호반베르디움', 'DMC호반베르디움더포레3단지')
changeWord('고양 향동지구 B-4블록 호반베르디움', 'DMC호반베르디움더포레4단지')
changeWord('고양향동 A-3블록 공공분양', '엘에이치A3(두산)아파트')
changeWord('고양향동지구 A-2블록 중흥S-클래스', 'DMC 중흥S-클래스 더 센트럴')
changeWord('일산자이 2차', '일산자이2차')
changeWord('킨텍스 원시티 M1BL', '킨텍스원시티 1블럭')
changeWord('킨텍스 원시티 M2BL', '킨텍스원시티 2블럭')
changeWord('킨텍스 원시티 M3BL', '킨텍스원시티 3블럭')
changeWord('고양 일산 센트럴 아이파크', '일산센트럴아이파크')
changeWord('킨텍스꿈에그린', '킨텍스 꿈에그린')
changeWord('고양 일산 에듀포레 푸르지오', '일산에듀포레푸르지오')
changeWord('과천 래미안 센트럴스위트', '래미안 센트럴스위트')
changeWord('과천 센트럴 파크 푸르지오 써밋', '센트럴파크푸르지오써밋')
changeWord('과천 위버필드', '과천위버필드')
changeWord('광명역 태영 데시앙', '유-플래닛 태영데시앙')
changeWord('광명 철산 센트럴 푸르지오', '철산센트럴푸르지오')
changeWord('구리 갈매역 아이파크(보금자리)', '갈매역아이파크')
changeWord('더샵 나인힐스', '갈매 더샵 나인힐스')
changeWord('e편한세상 구리수택', 'e편한세상수택센트럴파크')
changeWord('군포 당동 동도센트리움', '군포당동 동도센트리움 아파트')
changeWord('군포송정 B1 금강펜테리움 센트럴파크', '금강펜테리움센트럴파크1차')
changeWord('군포송정B2블록 금강펜테리움센트럴파크', '금강펜테리움센트럴파크2차')
changeWord('군포송정C1 금강펜테리움센트럴파크3차', '금강펜테리움센트럴파크3차')
changeWord('군포송정S1블록 공공분양', '엘에이치송정마을')
changeWord('걸포북변역 우방 아이유쉘', '한강파크뷰우방아이유쉘')
changeWord('김포 고촌 NEW CITY', '김포뉴시티')
changeWord('김포 캐슬앤파밀리에시티 1단지', '캐슬앤파밀리에시티 1단지')
changeWord('김포한강 AC-03블록 반도유보라5차', '한강신도시 반도유보라5차')
changeWord('김포한강 cc-01 블록 반도유보라4차', '한강신도시 반도유보라 4차')
changeWord('김포한강 AC-04블록 모아엘가2차', '한강신도시 모아엘가2차')
changeWord('한강신도시푸르지오3차', '한강신도시 3차 푸르지오')
changeWord('김포한강 Ab-18블록 호반베르디움', '호반베르디움 더 레이크 2차')
changeWord('김포한강 Ab-19블록 호반베르디움', '호반베르디움 더 레이크 2차')
changeWord('김포한강 동일스위트ThePark1단지', '한강 동일스위트 The Park View 1단지')
changeWord('김포한강 동일스위트ThePark2단지', '한강동일스위트 The Park View 2단지')
changeWord('김포 한강신도시 레이크 에일린의 뜰', '한강신도시레이크에일린의뜰')
changeWord('김포한강신도시 반도유보라3차', '한강신도시 반도유보라3차')
changeWord('김포사우2지구 김포사우 아이파크', '김포사우아이파크')
changeWord('김포한강 금호어울림1단지', '한강금호어울림 1단지')
changeWord('김포한강 금호어울림2단지', '한강금호어울림 2단지')
changeWord('김포한강 삼정그린코아 더베스트', '김포한강 삼정그린코아더베스트')
changeWord('김포한강신도시 리버 에일린의 뜰', '한강신도시리버에일린의뜰')
changeWord('한강신도시운양역푸르지오', '한강신도시 운양푸르지오')
changeWord('김포한강신도시Ab-12 이랜드 타운힐스', '김포한강 이랜드 타운힐스')
changeWord('김포한강Ac20블록 반도유보라6차', '한강신도시 반도유보라 6차')
changeWord('김포 한강신도시 e편한세상 캐널시티', 'e편한세상 캐널시티')
changeWord('김포풍무 푸르지오 센트레빌', '풍무 푸르지오')
changeWord('아너스빌위즈', '수원 아너스빌 위즈')
changeWord('수원역 푸르지오 자이', '수원역푸르지오자이')
changeWord('수원 인계동 동문굿모닝힐', '수원 인계동 동문굿모닝힐 아파트')
changeWord('수원 인계 한양수자인', '인계한양수자인')
changeWord('시흥 은계지구 우미 린B-3블록', '시흥은계우미린레이크')
changeWord('시흥장현 B4블록 제일풍경채 센텀', '제일풍경채 센텀')
changeWord('e편한세상 시흥', '이편한세상시흥센텀하임')
changeWord('목감 레이크 푸르지오(A-7블록)', '목감레이크 푸르지오')
changeWord('시흥목감지구 B1블록 중흥S클래스', '목감중흥에스-클래스레이크힐스')
changeWord('시흥 은계지구 우미 린C-1블록', '시흥은계우미린더퍼스트')
changeWord('시흥은계지구 B-5블록 한양수자인', '시흥은계한양수자인더클래스')
changeWord('시흥은계지구 복-1 호반 써밋플레이스', '시흥은계호반써밋플레이스')
changeWord('시흥장현 B5블록 제일풍경채 에듀', '제일풍경채 에듀')
changeWord('시흥장현 C-2BL 모아미래도에듀포레', '시흥능곡역 모아미래도 에듀포레')
changeWord('시흥장현 C1블록 리슈빌', '시흥능곡역 장현리슈빌 퍼스트클래스')
changeWord('장현 호반 써밋', '장현호반써밋')
changeWord('시흥목감Ｂ－２신안인스빌', '신안인스빌')
changeWord('시흥목감C1블럭 호반베르디움', '시흥목감C1BL 호반베르디움 더 클래스')
changeWord('시흥목감C1블럭 호반베르디움', '시흥목감C1BL 호반베르디움 더 클래스')
changeWord("오산시티자이 2단지", "오산시티자이1차 1단지")
df.loc[(df.번지=='698') & (df.단지명=='오산시티자이1차 2단지'),'단지명'] = '오산시티자이1차 2단지'
df.replace({'시군구':'경기도 평택시 고덕면'},{'시군구':'경기도 평택시 고덕면 여염리'}, inplace=True)
changeWord("고잔 롯데캐슬 골드파크", "고잔롯데캐슬골드파크")
changeWord("안산 센트럴 푸르지오", "센트럴푸르지오")
changeWord("힐스테이트 중앙", "힐스테이트중앙")
changeWord("레이크타운푸르지오", "안산레이크타운푸르지오")
changeWord("e편한세상 선부 ", "e편한세상선부")
changeWord("e편한세상선부역어반스퀘어", "이편한세상 선부역 어반스퀘어")
changeWord("안산 메트로타운 푸르지오 힐스테이트", "안산 메트로타운 푸르지오힐스테이트")
changeWord("안산 천년가 리더스카이", "천년가 리더스카이")
changeWord("초지역 메이저타운 푸르지오 에코단지", "초지역메이저타운푸르지오에코단지")
changeWord("안산 롯데캐슬 더퍼스트", "롯데캐슬더퍼스트")
changeWord("초지역 메이저타운 푸르지오 메트로단지", "초지역메이저타운푸르지오메트로단지")
changeWord("초지역 메이저타운 푸르지오 파크단지", "초지역메이저타운푸르지오파크단지")
changeWord("건건동 영무예다음", "영무예다음")
changeWord("그랑시티자이(안산 사동 90블록)", "그랑시티자이")
changeWord("그랑시티자이2차", "그랑시티자이 2차")
changeWord("안산 파크 푸르지오", "파크푸르지오")
changeWord("안양 신평촌 일성트루엘 파크", "신평촌일성트루엘파크")
changeWord("안양 호계 대성유니드", "평촌대성유니드")
changeWord("평촌 더샵 아이파크", "평촌더샵아이파크")
changeWord("평촌 어바인 퍼스트", "평촌어바인퍼스트")
changeWord("래미안 안양 메가트리아", "래미안안양메가트리아")
changeWord("안양 명학역 유보라 더 스마트", "안양 명학역유보라 더스마트")
changeWord("한양수자인 안양역", "안양역한양수자인리버파크")
changeWord("오산세교택지개발지구 오산 세교 자이", "오산세교자이")
changeWord("오산시티자이１단지", "오산시티자이1차 1단지")
changeWord("오산시티자이 2차", "오산시티자이2차")
changeWord("오산대역 더샵 센트럴시티", "더샵 오산센트럴")
changeWord("오산 센트럴 푸르지오", "오산센트럴푸르지오")
changeWord("오산원동 한양수자인", "오산원동 한양수자인아파트")
changeWord("e편한세상 오산세교", "e편한세상오산세교")
changeWord("오산세교e편한세상", "e편한세상오산세교")
changeWord("기흥역 지웰 푸르지오", "기흥역지웰푸르지오")
changeWord("용인 기흥역 더샵", "기흥역더샵")
changeWord("용인 기흥역 파크 푸르지오 (3-3BL)", "기흥역 파크 푸르지오")
changeWord("힐스테이트 기흥 ", "힐스테이트기흥")
changeWord("용인 구성 지음재 아파트", "용인구성지음재")
changeWord("용인구성역효성해링턴플레이스", "구성역효성해링턴플레이스")
changeWord("용인 보라 효성해링턴 플레이스", "용인보라효성해링턴플레이스")
changeWord("서천 2차 I PARK", "서천2차IPARK")
changeWord("용인 영통로 효성해링턴 플레이스", "영통로 효성해링턴 플레이스 1차")
changeWord("용인 기흥 우방아이유쉘", "용인기흥 우방아이유쉘")
changeWord("용인신갈 신흥덕 롯데캐슬 레이시티", "신흥덕 롯데캐슬 레이시티")
changeWord("기흥 힉스유타워 ", "힉스유타워")
changeWord("용인 기흥 효성해링턴 플레이스", "용인기흥 효성해링턴플레이스")
changeWord("용인 신동백 동도센트리움 도시형생활주택", "신동백 동도센트리움")
changeWord("동천 더샵 이스트포레", "더샵 동천이스트포레")
changeWord("분당 수지 루체스타", "루체스타")
changeWord("용인 동천 자이", "동천자이")
changeWord("용인 동천자이 2차", "동천자이2차")
changeWord("용인 동천파크자이", "동천파크자이")
changeWord("레이크포레 수지", "레이크포레")
changeWord("용인 상현 더샵 파크사이드", "상현 더샵 파크사이드")
changeWord("용인 수지 동도센트리움 도시형생활주택", "용인수지동도센트리움")
changeWord("용인수지휴엔하임", "용인수지 휴엔하임 아파트")
changeWord("성복역 롯데캐슬 파크나인", "성복역 롯데캐슬 파크나인1차")
changeWord("용인 성복 서희스타힐스 애비뉴", "성복역 서희스타힐스")
changeWord("용인 성복역 KCC스위첸", "성복역KCC스위첸")
changeWord("용인 성복역 롯데캐슬 골드타운", "성복역 롯데캐슬 골드타운")
changeWord("수지 신봉 동도센트리움", "수지신봉동도센트리움")
changeWord("수지 풍덕천 한샘 더 랜드마크", "수지 풍덕천 한샘 더랜드마크")
changeWord("이편한세상 수지", "e편한세상수지")
changeWord("용인 수지 파크 푸르지오", "수지 파크 푸르지오")
changeWord("용인고림지구2차 양우내안애 에듀퍼스트", "양우내안애에듀퍼스트")
changeWord("용인 고림지구 양우내안애", "양우내안애에듀파크")
changeWord("용인 웰메이드시티337", "용인웰메이드시티")
changeWord("유메이드", "용인유메이드")
changeWord("e편한세상 용인 한숲시티 2BL", "e편한세상용인한숲시티2단지")
changeWord("e편한세상 용인 한숲시티 3BL", "e편한세상용인한숲시티3단지")
changeWord("e편한세상 용인 한숲시티 4BL", "e편한세상용인한숲시티4단지")
changeWord("e편한세상 용인 한숲시티 5BL", "e편한세상용인한숲시티5단지")
changeWord("e편한세상 용인 한숲시티 6BL", "e편한세상용인한숲시티6단지")
changeWord("용인 역북 골드클래스", "골드클래스")
changeWord("용인 역북 명지대역 동원로얄듀크", "용인명지대역동원로얄듀크")
changeWord("용인 역북 신원 아침도시", "역북신원아침도시")
changeWord("용인 역북 우미린 센트럴파크", "우미린센트럴파크")
changeWord("에이스카운티", "에이스카운티아파트")
changeWord("의왕 장안지구 파크 푸르지오", "의왕파크푸르지오 1차")
changeWord("의왕서해그랑블", "의왕서해그랑블(블루스퀘어)")
changeWord("의왕 포일 센트럴 푸르지오", "인덕원 푸르지오 엘센트로")
changeWord("의왕백운밸리 효성해링턴플레이스 B블록", "의왕백운 해링턴 플레이스 1단지")
changeWord("의왕백운밸리 효성해링턴플레이스C1블록", "의왕백운 해링턴 플레이스 2단지")
changeWord("의왕백운밸리 효성해링턴플레이스C2블록", "의왕백운 해링턴 플레이스 3단지")
changeWord("의왕백운밸리 효성해링턴플레이스C3블록", "의왕백운 해링턴 플레이스 4단지")
changeWord("의왕백운밸리 효성해링턴플레이스C4블록", "의왕백운 해링턴 플레이스 5단지")
changeWord("힐스테이트 녹양역", "힐스테이트녹양역")
changeWord("의정부 민락2 반도유보라 아이비파크", "반도유보라아이비파크")
changeWord("의정부민락2지구 B11 대광로제비앙", "대광 로제비앙 포레스트")
changeWord("의정부 민락2지구 우미린", "의정부민락우미린더스카이")
changeWord("의정부민락2 호반베르디움 B14,B15", "호반베르디움1차")
changeWord("의정부민락２Ｂ１２호반베르디움", "민락2지구18단지호반베르디움")
changeWord("의정부 고산 대광로제비앙", "대광로제비앙 더 퍼스트")
changeWord("승윤 노블리안시티", "승윤노블리안시티")
changeWord("승윤노블리안시티2", "승윤노블리안시티II")
changeWord("한원힐트리움 더테라스", "의정부 한원힐트리움-더테라스")
changeWord("의정부 롯데캐슬 골드파크 1BL", "의정부 롯데캐슬 골드파크 1단지")
changeWord("의정부 롯데캐슬 골드파크 2BL", "의정부 롯데캐슬 골드파크 2단지")
changeWord("의정부 장암 더샵 ", "장암더샵")
changeWord("의정부 르네상스시티", "르네상스시티")
changeWord("운정신도시 아이파크 ", "운정신도시 IPARK")
changeWord("파주 운정 화성파크드림", "운정 화성파크드림 시그니처")
changeWord("파주 힐스테이트 운정", "힐스테이트운정")
changeWord("파주 문산역 동문굿모닝힐", "파주문산역1차 동문 굿모닝힐")
changeWord("파주 운정신도시 롯데캐슬 파크타운", "운정롯데캐슬파크타운1차")
changeWord("파주 운정신도시 롯데캐슬 파크타운2차", "롯데캐슬 파크타운Ⅱ")
changeWord("파주 효성해링턴 플레이스", "효성해링턴플레이스")
changeWord("미사강변도시 A18BL", "미사강변 센텀팰리스")
changeWord("미사강변푸르지오1차 A30블럭", "미사강변푸르지오")
changeWord("하남미사강변C2블럭 호반 써밋플레이스", "미사강변 호반 써밋")
changeWord("하남미사 A1블럭 미사강변리버뷰자이", "미사강변리버뷰자이")
changeWord("e편한세상 미사 ", "이편한세상미사")
changeWord("미사강변 동원로얄듀크", "미사강변동원로얄듀크")
changeWord("하남미사 A32블럭 신안인스빌", "신안인스빌")
changeWord("하남 힐즈파크 푸르지오 1BL", "하남힐즈파크푸르지오 1단지")
changeWord("하남 힐즈파크 푸르지오 2BL", "하남힐즈파크푸르지오 2단지")
changeWord("하남 힐즈파크 푸르지오 3BL", "하남힐즈파크푸르지오 3단지")
changeWord("하남미사강변도시 A33블록 제일풍경채", "미사강변센트럴풍경채")
changeWord("위례 신안인스빌 아스트로", "위례신도시신안인스빌아스트로")
changeWord("위례에코앤롯데캐슬(A3-8)", "위례롯데캐슬")
changeWord("위례그린파크푸르지오(A3-9)", "위례그린파크 푸르지오")
changeWord("엠코타운2차 센트로엘(A3-6A)", "위례신도시엠코타운센트로엘")
changeWord("화성기산 SK VIEW PARK 2차", "에스케이뷰파크2차")
changeWord("양우내안에(남양뉴타운B3)", "남양뉴타운 양우내안애 1차")
changeWord("화성남양뉴타운 B2블록 양우내안애２차", "양우내안애2차에듀타운아파트")
changeWord("화성 남양뉴타운 동광뷰엘", "동광뷰엘")
changeWord("화성 남양 B12블록 시티프라디움", "시티프라디움1차아파트")
changeWord("화성 남양뉴타운 B1블록 시티프라디움", "시티프라디움1차아파트")
changeWord("송산그린시티 세영리첼 에듀파크", "세영리첼 에듀파크")
changeWord("신동탄파크자이", "서동탄역파크자이")
changeWord("신동탄파크자이 2차", "서동탄역파크자이2차")
changeWord("e편한세상 동탄 A45", "이편한세상동탄")
changeWord("금강펜테리움 센트럴파크(A46블럭)4차", "동탄 금강펜테리움 센트럴파크Ⅳ")
changeWord("동탄2신도시 2차 동원로얄듀크", "동탄동원로얄듀크2차")
changeWord("동탄2신도시 A47블록 한신휴플러스", "한신더휴")
changeWord("동탄2신도시 힐스테이트 동탄", "힐스테이트 동탄")
changeWord("동탄２신도시 호반베르디움 22단지", "동탄2신도시 호반베르디움 22단지")
changeWord("고덕 하늘채 시그니처", "고덕하늘채 시그니처")
changeWord("평택 고덕파라곤", "고덕국제신도시파라곤")
changeWord("평택 더샵 센트럴파크 1BL", "The Sharp 지제역 센트럴파크 1BL")
changeWord("평택 더샵 센트럴파크 2BL", "The Sharp 지제역 센트럴파크 2BL")
changeWord("평택자이더 익스프레스4BL", "평택센트럴자이 4단지")
changeWord("평택자이더 익스프레스5BL", "평택센트럴자이 5단지")
changeWord("평택소사벌 B-1 푸르지오", "평택소사벌푸르지오")
changeWord("힐스테이트 평택 3차", "힐스테이트평택3차(3-1블록)")
changeWord("힐스테이트 평택2차", "힐스테이트 2차(세교지구 2-1BL)")
changeWord("힐스테이트 평택2차 (세교도시개발2-1)", "힐스테이트 2차(세교지구 2-1BL)")
changeWord("힐스테이트 평택(세교도시개발1-2블럭)", "힐스테이트 평택 1차")
changeWord("효성해링턴 플레이스 1BL", "평택효성해링턴플레이스1단지")
changeWord("효성해링턴 플레이스 2BL", "평택효성해링턴플레이스2단지")
changeWord("평택 송탄역 서희스타힐스", "송탄역 서희스타힐스")
changeWord("평택 신장동 메디슨스퀘어3", "메디슨스퀘어3차")
changeWord("평택송담힐스테이트", "힐스테이트 송담")
changeWord("평택 송담지구 79-1블록 지엔하임", "송담 지엔하임1차")
changeWord("평택 송담지구 80-2블록 지엔하임", "송담 지엔하임2차")
changeWord("e 편한세상 평택용이(신흥지구1BL)", "e편한세상 평택용이 1단지")
changeWord("e 편한세상 평택용이(신흥지구 2BL)", "e편한세상 평택용이 2단지")
changeWord("비전아이파크 평택", "비전아이파크평택")
changeWord("서정라페온빌", "서정라페온빌3차")
changeWord("평택 장당 제일풍경채3차", "제일풍경채장당센트럴")
changeWord("소사벌더샵", "소사벌더샵마스터뷰")
changeWord("평택 소사벌 우미린 레이크파크", "우미린레이크파크")
changeWord("평택소사벌지구 B11블럭 호반베르디움", "평택소사벌 호반베르디움")
changeWord("평택청북지구B-11 브라운스톤 청북", "브라운스톤청북")
changeWord("평택 지제역 동문굿모닝힐 맘시티 A1", "평택지제역동문굿모닝힐맘시티1단지")
changeWord("평택 지제역 동문굿모닝힐 맘시티 A2", "평택지제역동문굿모닝힐맘시티2단지")
changeWord("평택 지제역 동문굿모닝힐 맘시티 A4", "평택지제역동문굿모닝힐맘시티4단지")
changeWord("더 맥심 험프리스", "맥심험프리스")
changeWord("라페온빌", "라페온빌1차")
changeWord("평택역 동우 자인채 센트럴", "동우자인채센트럴")
changeWord("평택역 코아루두드림", "평택역코아루두드림")
changeWord("평택 골든포트", "평택포승 골든포트")
changeWord("평택 비전 레이크 푸르지오", "평택비전레이크푸르지오")
changeWord("평택비전푸르지오(A-3블록)", "평택비전푸르지오")
changeWord('남양주다산신도시 아이파크', '다산아이파크')
changeWord('다산신도시 금강펜테리움 리버테라스2차', '다산펜테리움리버테라스Ⅱ')
changeWord('다산신도시 금강펜테리움 리버테라스 B4', '다산 펜테리움 리버테라스 Ⅰ')
changeWord('다산아이파', '다산아이파크')
changeWord('다산지금지구B3 신안인스빌 퍼스트포레', '다산신안인스빌퍼스트포레')
changeWord('다산지금B5 반도유보라 메이플타운2.0', '다산반도유보라메이플타운2.0')
changeWord('지금 힐스테이트', '힐스테이트 황금산')
changeWord('남양주 라온프라이빗（1-1블럭）', '남양주 라온프라이빗 1단지')
changeWord('남양주 라온프라이빗（2-1블럭）', '남양주 라온프라이빗 2단지')
changeWord('남양주 라온프라이빗（1-2블럭）', '남양주 라온프라이빗 1단지')
changeWord('남양주 라온프라이빗（3-1블럭）', '남양주 라온프라이빗 3단지')
changeWord('남양주 라온프라이빗（4-1블럭）', '남양주 라온프라이빗 4단지')
changeWord('남양주 마석 푸르지오', '마석힐즈파크푸르지오')
changeWord('남양주창현도뮤토', '남양주 창현 도뮤토')
changeWord('부천 e편한세상 온수역', '이편한세상 온수역')
changeWord('부천 한신더휴메트로', '부천 한신더휴 메트로')
changeWord('송내역파인푸르지오1단지', '송내역 파인 푸르지오 1단지')
changeWord('송내역파인푸르지오2단지', '송내역 파인 푸르지오 2단지')
changeWord('부천 심곡동 우림필유 ', '부천우림필유에듀파크')
changeWord('동도센트리움', '동도 센트리움')
changeWord('부천옥길 호반베르디움', '옥길호반베르디움')
changeWord('부천옥길자이（보금자리주택）', '부천옥길자이')
changeWord('부천옥길지구 제일풍경채(보금자리) ', '제일풍경채 제이드카운티2단지')
changeWord('부천옥길지구 한신휴플러스(보금자리)', '한신더휴 제이드카운티1단지')
changeWord('부천 중동 센트럴파크 푸르지오', '센트럴파크 푸르지오')
changeWord('부천 중동 효성해링턴 플레이스', '부천 중동 해링턴플레이스')
changeWord('판교 퍼스트힐 푸르지오 A2BL', '판교퍼스트힐푸르지오A2')
changeWord('산성역 포레스티아', '산성역포레스티아')
changeWord('위례 센트럴푸르지오(A2-9)', '위례센트럴 푸르지오')
changeWord('자연앤래미안e편한세상(A2-11)', '위례자연앤래미안이편한세상')
changeWord('위례지구 A2-8블럭 호반베르디움', '위례호반베르디움')
changeWord('위례아트리버 푸르지오1단지(C2-2)', '위례아트리버푸르지오1단지')
changeWord('위례아트리버푸르지오2단지C2-3', '위례아트리버푸르지오2단지')
changeWord('위례C2-1BL 보미리즌빌 주상복합', '위례보미리즌빌')
changeWord('위례신도시 A2-3BL 위례자이', '위례자이')
changeWord('한양수자인 성남마크뷰', '한양수자인성남마크뷰')
changeWord('수원호매실호반베르디움(B5블록)', '호반베르디움 더 센트럴')
changeWord('수원호매실호반베르디움(B9블록)', '호반베르디움 더 센트럴')
changeWord('수원 오목천동 서희스타힐스', '오목천역 서희스타힐스 아파트')
changeWord('수원 호매실 금호어울림 에듀포레', '호매실금호어울림에듀포레')
changeWord('한양수자인 파크원', '한양수자인파크원')
changeWord('e편한세상영통2차1단지', '이편한세상2차1단지')
changeWord('e편한세상영통2차2단지', '이편한세상2차2단지')
changeWord('영통SK VIEW', '영통 SK VIEW')
changeWord('수원 영통 아이파크캐슬 1블록', '영통아이파크캐슬1단지')
changeWord('수원 영통 아이파크캐슬 2블록', '영통아이파크캐슬2단지')
changeWord('힐스테이트영통', '힐스테이트 영통')
changeWord('수원 영통 라온프라이빗', '영통 라온프라이빗')
changeWord('수원 광교 C-2 블럭 중흥S클래스', '광교중흥에스클래스')
changeWord('힐스테이트광교(D3블럭)', '힐스테이트광교')
changeWord('호반베르디움센트럴포레', '호반베르디움 센트럴포레')
changeWord('e편한세상 화성', 'e편한세상 반월 나노시티역')
changeWord('e편한세상 신봉담 공공분양', '이편한세상신봉담')
changeWord('봉담역 동문 굿모닝힐', '호수마을동문굿모닝힐')
changeWord('봉담2차우방아이유쉘', '우방아이유쉘2단지')
changeWord('봉담우방아이유쉘', '우방아이유쉘1단지')
changeWord('봉담 한신더휴 에듀파크', '봉담한신더휴에듀파크')
changeWord('동탄2 A73블록 사랑으로 부영', '더레이크시티부영4단지')
changeWord('동탄2 A74블록 사랑으로 부영', '더레이크시티부영5단지')
changeWord('동탄2 A75블록 사랑으로 부영', '더레이크시티부영6단지')
changeWord('동탄2 A79 반도유보라 아이비파크10.0', '레이크힐 반도유보라 아이비파크 10.2')
changeWord('동탄2 A80 반도유보라 아이비파크10.0', '그린힐 반도유보라 아이비파크 10(1단지)')
changeWord('동탄2신도시 A70블록 사랑으로 부영', '더레이크시티부영1단지')
changeWord('동탄2신도시 A71블록 사랑으로 부영', '더레이크시티부영2단지')
changeWord('동탄2신도시 A72블록 사랑으로 부영', '더레이크시티부영3단지')
changeWord('송산그린EAA10BL 모아미래도 에듀포레', '모아미래도 에듀포레')
changeWord('화성송산그린시티ＥＡＡ２블럭ＥＧｔｈｅ１', '송산그린시티 이지더원 레이크뷰 아파트')
changeWord('동탄2신도시 하우스디 더 레이크', '동탄2하우스디더레이크')
changeWord('화성 안녕동 우방아이유쉘', '안녕동우방아이유쉘')
changeWord('경남아너스빌(A101블럭)', '동탄역 경남아너스빌')
changeWord('동탄2신도시 1차 동원로얄듀크', '동탄역동원로얄듀크1차')
changeWord('동탄에듀밸리사랑으로부영', '동탄에듀밸리사랑으로부영아파트')
changeWord('동탄파크자이(A8블럭)', '동탄파크자이')
changeWord('금강펜테리움 센트럴파크 동탄(A-39)', '동탄2신도시 금강펜테리움 센트럴파크Ⅰ')
changeWord('동탄역파라곤（주）', '동탄역 파라곤')
changeWord('반도유보라아이비파크2차(A13블럭)', '동탄역반도유보라아이비파크2.0')
changeWord('동탄2신도시 3차 푸르지오', '동탄파크푸르지오')
changeWord('동탄2 A35블록 중흥S-클래스', '중흥에스클래스에코벨리')
changeWord('동탄2신도시 3차 동원로얄듀크 비스타', '동탄역동원로얄듀크비스타3차')
changeWord('동탄2신도시 A-37 반도유보라 5.0', '동탄역 반도유보라 아이비파크5.0')
changeWord('반도유보라아이비파크3.0(A-38블럭)', '반도유보라아이비파크3')
changeWord('화성동탄2 반도유보라 아이비파크8.0', '동탄역 반도유보라 아이비파크 8.0')
changeWord('동탄 자이파밀리에 ', '동탄호수자이파밀리에')
changeWord('동탄2 A96블록 제일풍경채 에듀파크', '제일풍경ㄹ채에듀앤파크')
changeWord('동탄2 금호어울림 레이크 ', '금호어울림레이크')
changeWord('호반베르디움6차(A97)', '동탄2신도시 호반베르디움 33단지')
changeWord('화성동탄2 반도유보라 아이비파크9.0', '레이크 반도유보라 아이비파크 9.0')
changeWord('동탄2지구 금성백조예미지(A-17)', '동탄역시범예미지아파트')
changeWord('동탄꿈에그린프레스티지(A-21블록)', '동탄역 시범한화 꿈에그린 프레스티지')
changeWord('동탄역 더삽 센트럴시티)', '더샵센트럴시티')
changeWord('반도유보라 아이비파크4.0(C15블럭)', '시범 반도유보라 아이비파크4.0')
changeWord('반도유보라아이비파크(A-18)', '동탄역 시범반도유보라아이비파크1.0')
changeWord('센트럴푸르지오(A29블럭)', '동탄2신도시 센트럴 푸르지오')
changeWord('신안인스빌리베라1차(A32블럭)', '동탄2신도시신안인스빌 1차')
changeWord('신안인스빌리베라2차(A26)', '신안인스빌리베라2차')
changeWord('호반베르디움더클래스(A30블럭)', '동탄2신도시 호반베르디움 더클래스')

(1017, 2)

(1017, 2)

(1017, 2)

(1017, 2)

(1017, 2)

...

(759, 2)

(758, 2)

(757, 2)

(756, 2)

(755, 2)

### 송도 더샵 센트럴시티 수정

In [23]:
central = df[df['단지명']=="송도 더샵 센트럴시티"].index
df = df.drop(central)
df[df.단지명=="송도 더샵 센트럴시티"]

,시군구,번지,단지명,전용면적(㎡),거래금액(만원),층,계약년월,계약일,분/입구분,해제사유발생일,지역,id


In [24]:
centralcity = pd.read_csv("data/etc/centralcity.csv")
centralcity.head()

,시군구,번지,단지명,전용면적(㎡),거래금액(만원),층,계약년월,계약일,분/입구분,해제사유발생일,거래유형,중개사소재지
0,인천광역시 연수구 송도동,190-4,송도 더샵 센트럴시티,72.99,"39,760",20,201701,3,분,NaN,-,-
1,인천광역시 연수구 송도동,190-4,송도 더샵 센트럴시티,84.97,"46,020",33,201701,4,분,NaN,-,-
2,인천광역시 연수구 송도동,190-4,송도 더샵 센트럴시티,59.97,"35,185",11,201701,5,분,NaN,-,-
3,인천광역시 연수구 송도동,190-4,송도 더샵 센트럴시티,59.99,"35,120",28,201701,6,분,NaN,-,-
4,인천광역시 연수구 송도동,190-4,송도 더샵 센트럴시티,72.99,"39,015",7,201701,9,분,NaN,-,-


In [26]:
df = pd.concat([df, centralcity])
df.shape

(214592, 14)

In [27]:
# 서울특별시
changeWord('래미안 강남포레스트', '개포래미안포레스트')
changeWord('주공2', '래미안블레스티지')
changeWord('래미안대치청실', '래미안대치팰리스')
changeWord('삼성래미안(진달래1차)', '삼성래미안')
changeWord('래미안 루체하임 ', '래미안 개포 루체하임')
changeWord('강일리버파크', '강일리버파크5단지')
changeWord('고덕래미안힐스테이트', '고덕래미안 힐스테이트아파트')
changeWord('래미안 명일역 솔베뉴', '래미안솔베뉴')
changeWord('삼익그린1차', '래미안솔베뉴')
changeWord('고덕 센트럴 IPARK ', '고덕센트럴아이파크')
changeWord('고덕 아르테온 ', '고덕아르테온')
changeWord('힐스테이트 암사', '힐스테이트암사')
changeWord('강동역 신동아파밀리에', '강동역신동아파밀리에')
changeWord('래미안 강동팰리스', '래미안강동팰리스')
changeWord('미아9-1 꿈의숲 효성해링턴 플레이스', '꿈의숲해링턴플레이스')
changeWord('방화역 메이빌 아파트', '방화메이빌')
changeWord('강서힐스테이트(화곡3주구)', '강서힐스테이트')
changeWord('e편한세상 서울대입구', 'e편한세상 서울대입구 1단지')
changeWord('한양아이클래스(도시형)', '서울대입구한양아이클래스')
changeWord('강변SK VIEW', '강변에스케이뷰')
changeWord('SH 항동 공공주택지구2단지 공공분양', '하버라인2단지')
changeWord('SH 항동 공공주택지구3단지', '하버라인3단지')
changeWord('SH 항동 공공주택지구4단지 공공분양', '하버라인4단지')
changeWord('서울항동공공주택지구1BL 중흥S클래스', '항동중흥에스클래스베르데카운티')
changeWord('가산지웰에스테이트ⅠⅠ', '가산 지웰 에스테이트 2차')
changeWord('롯데캐슬골드파크3차', '금천 롯데캐슬 골드파크 3차')
changeWord('롯데캐슬골드파크2차', '금천롯데캐슬골드파크2차201동202동')
changeWord('6.7 디벨리움 도시형 생활주택', '6.7디벨리움')
changeWord('상계역 센트럴 푸르지오 ', '노원센트럴 푸르지오')
changeWord('꿈의숲SK뷰', '꿈의숲 SK VIEW')
changeWord('녹천역두산위브', '녹천역두산위브아파트')
changeWord('답십리래미안위브', '래미안위브')
changeWord('답십리역 한일휴니스빌 도시형생활주택', '한일노벨리아')
changeWord('래미안 답십리 미드카운티 ', '래미안 미드카운티')
changeWord('서울 청계 와이즈 노벨리아', '청계와이즈노벨리아')
changeWord('현대 썬앤빌 청계 도시형생활주택', '현대썬앤빌청계')
changeWord('동대문푸르지오시티', '동대문푸르지오시티(도시형생활주택)')
changeWord('현대 썬앤빌 601', '현대썬앤빌601')
changeWord('휘경 SK VIEW ', '휘경 SK뷰')
changeWord('휘경제이스카이', '휘경 제이스카이 아파트')
changeWord('사당 롯데캐슬 골든포레 ', '롯데캐슬골든포레')
changeWord('사당반석블레스', '반석블레스')
changeWord('e편한세상 상도 노빌리티', '편한세상상도노빌리티')
changeWord('두산위브지역주택조합(두산위브트레지움)', '두산위브트레지움2차')
changeWord('상도 휴엔하임(도시형)', '상도 휴엔하임')
changeWord('상도역 롯데캐슬', '상도역 롯데캐슬 파크엘')
changeWord('보라매 e-편한세상', '보라매 e편한세상')
changeWord('흑석뉴타운 롯데캐슬 에듀포레', '롯데캐슬에듀포레')
changeWord('흑석뉴타운 아크로 리버하임', '아크로리버하임')
changeWord('마포 한강 IPARK', '마포한강아이파크')
changeWord('신촌숲 IPARK ', '신촌숲아이파크')
changeWord('마포 애오개 아이파크 도시형생활주택', '애오개아이파크')
changeWord('마포아현IPARK', '아현아이파크')
changeWord('서울 마포자이 3차', '마포자이3차')
changeWord('e편한세상마포3차', '편한세상마포리버파크')
changeWord('마포한강2차 푸르지오', '마포한강2차푸르지오')
changeWord('래미안 마포 웰스트림', '래미안 웰스트림')
changeWord('DMC 2차 IPARK', '센트럴아이파크')
changeWord('상암DMC 엘가 도시형생활주택', '디엠씨엘가')
changeWord('e편한세상 신촌', 'e편한세상신촌')
changeWord('북한산 더샵 ', '북한산더샵')
changeWord('북한산 두산위브１차', '북한산두산위브1차')
changeWord('홍제금호어울림', '금호어울림')
changeWord('반포 래미안 아이파크', '반포래미안아이파크')
changeWord('아크로리버 파크', '아크로리버파크')
changeWord('래미안 서초 에스티지S ', '래미안서초에스티지')
changeWord('마제스타시티 힐스테이트서리풀', '마제스타시티(힐스테이트서리풀101,102동)')
changeWord('신반포센트럴자이 ', '반포센트럴자이')
changeWord('아크로 리버뷰', '아크로리버뷰신반포')
changeWord('E 편한세상 신금호', '이편한세상금호파크힐스')
changeWord('힐스테이트 금호', '힐스테이트 서울숲 리버')
changeWord('텐즈힐', '텐즈힐(2구역)')
changeWord('옥수 이편한세상 파크힐스', '옥수파크힐스101동~116동')
changeWord('왕십리뉴타운1구역', '텐즈힐아파트(1단지)')
changeWord('왕십리뉴타운3구역 센트라스1,2차', '센트라스아파트')
changeWord('서울 숲 리버뷰 자이', '서울숲리버뷰자이')
changeWord('래미안 길음 센터피스 ', '래미안길음센터피스')
changeWord('성신여대역 솔하임', '성신여대역솔하임')
changeWord('꿈의숲 아이파크', '꿈의숲아이파크')
changeWord('래미안 장위 퍼스트하이', '래미안장위퍼스트하이')
changeWord('래미안 장위포레카운티', '래미안포레카운티')
changeWord('장위뉴타운 꿈의숲 코오롱하늘채', '꿈의숲코오롱하늘채')
changeWord('길음뉴타운 롯데캐슬 골든힐스', '롯데캐슬골든힐스')
changeWord('송파 헬리오시티', '헬리오시티')
changeWord('송파 파크하비오 푸르지오', '파크하비오')
changeWord('석촌호수 효성해링턴타워', '석촌호수효성해링턴타워')
changeWord('송파 두산위브 ', '송파두산위브')
changeWord('송파 오금지구 3블럭 호반베르디움', '송파호반베르디움더퍼스트')
changeWord('송파와이즈더샵(C1-4)', '송파와이즈더샵')
changeWord('송파힐스테이트(C1-1)', '힐스테이트송파위례')
changeWord('위례1차아이파크(C1-3)', '위례아이파크')
changeWord('위례2차아이파크(C1-2)', '위례2차아이파크')
changeWord('위례중앙푸르지오1단지 (BL5)', '위례중앙푸르지오1단지')
changeWord('위례중앙푸르지오2단지(C1-6블럭)', '위례중앙푸르지오2단지')
changeWord('잠실 올림픽 IPARK', '올림픽공원아이파크')
changeWord('목동 롯데캐슬 마에스트로', '목동롯데캐슬마에스트로')
changeWord('신정뉴타운1-1구역 아이파크위브', '목동센트럴 아이파크위브 1단지')
changeWord('래미안 목동아델리체', '래미안목동아델리체(1326)')
changeWord('목동파크자이 ', '목동파크자이(101동~108동)')
changeWord('당산역롯데캐슬프레스티지', '당산 롯데캐슬 프레스티지')
changeWord('e편한세상 보라매 2차', 'e편한세상 영등포 아델포레')
changeWord('신대림2차신동아파밀리에', '보라매 신동아 파밀리에')
changeWord('문래동 모아미래도', '문래동모아미래도아파트')
changeWord(' 래미안에스티움(신길7구역)', '래미안에스티움')
changeWord('보라매 SK VIEW', '보라매에스케이뷰')
changeWord('신길 뉴타운 IPARK', '신길센트럴아이파크')
changeWord('신길센트럴자이', '신길센트럴자이아파트')
changeWord('영등포 중흥S클래스', '중흥에스클래스')
changeWord('영등포 뉴타운 꿈에그린(포레나영등포)', '포레나영등포')
changeWord('영등포 아크로타워 스퀘어', '아크로타워스퀘어')
changeWord('용산더프라임', '용산 더프라임')
changeWord('래미안 용산', '래미안용산더센트럴')
changeWord('용산푸르지오써밋', '용산푸르지오써밋(420)')
changeWord('한남 해피트리', '한남해피트리')
changeWord('한남아이파크', '한남아이파크애비뉴')
changeWord('용산 롯데캐슬 센터포레', '용산롯데캐슬센터포레')
changeWord('효창 파크 KCC스위첸', '용산케이씨씨스위첸')
changeWord('래미안 베라힐즈 ', '래미안베라힐즈')
changeWord('힐스테이트 녹번', '힐스테이트녹번')
changeWord('롯데캐슬(불광4구역)', '불광롯데캐슬')
changeWord('DMC 롯데캐슬 더 퍼스트 ', 'DMC롯데캐슬더퍼스트')
changeWord('은평 신사 효성해링턴 플레이스', '효성해링턴플레이스')
changeWord('녹번역 e편한세상 캐슬', '녹번역e편한세상캐슬')
changeWord('백련산 해모로 ', '백련산해모로')
changeWord('서울 힐스테이트 백련산4차', '백련산힐스테이트 4차')
changeWord('은평뉴타운 꿈에그린 ', '은평뉴타운꿈에그린')
changeWord('은평뉴타운 솔하임(도시형)', '은평뉴타운솔하임')
changeWord('은평뉴타운 우물골(84-0)', '은평뉴타운 우물골(두산위브7단지)')
changeWord('은평뉴타운3지구B공구 2블록', '은평뉴타운 구파발(삼성래미안9단지)')
changeWord('은평뉴타운힐데스하임', '힐데스하임')
changeWord('은평스카이뷰자이', '은평 스카이뷰 자이')
changeWord('경희궁자이(2BL)', '경희궁자이(2단지)')
changeWord('경희궁자이(3BL)', '경희궁자이(3단지)')
changeWord('경희궁자이(4BL)', '경희궁자이(4단지)')
changeWord('효제포레스트힐시티', '포레스트힐시티')
changeWord('서울역 한라비발디 센트럴 아파트', '서울역한라비발디센트럴')
changeWord('서울 신당 KCC 스위첸', '신당 KCC 스위첸')
changeWord('신당역 솔하임 5차', '신당역솔하임')
changeWord('충무로 헤센스마트', '충무로헤센스마트')
changeWord('면목 라온프라이빗 ', '라온프라이빗')
changeWord('사가정 센트럴 아이파크', '사가정센트럴아이파크')
changeWord('쌍용더플래티넘용마산', '쌍용 더 플래티넘 용마산')
changeWord('한양수자인 사가정파크', '한양수자인사가정파크')
changeWord('ｅ편한세상 화랑대', '이편한세상 화랑대')
changeWord('금호어울림', '길음역금호어울림센터힐')


(754, 2)

개수가 1개 이상입니다.


,시군구,단지명
203109,인천광역시 서구 가좌동,주공2
21172,서울특별시 강남구 개포동,주공2


시군구를 입력해주세요인천광역시 서구 가좌동


(754, 2)

...

(613, 2)

(612, 2)

개수가 1개 이상입니다.


,시군구,단지명
4733,서울특별시 서대문구 홍제동,금호어울림
6812,서울특별시 성북구 돈암동,금호어울림


KeyboardInterrupt: Interrupted by user

In [125]:
# 일부 데이터 시군구 변경
df.loc[(df.시군구=='서울특별시 종로구 교남동') & (df.단지명 =='경희궁자이(2단지)'), '시군구'] = '서울특별시 종로구 홍파동'
df.loc[(df.시군구=='서울특별시 종로구 교남동') & (df.단지명 =='경희궁자이(3단지)'), '시군구'] = '서울특별시 종로구 평동'
df.loc[(df.시군구=='서울특별시 종로구 교남동') & (df.단지명 =='경희궁자이(4단지)'), '시군구'] = '서울특별시 종로구 교북동'

In [95]:
# 인천광역시
changeWord('인천 계양산 파크 트루엘', '계양산파크트루엘')
changeWord('계양코아루센트럴파크', '인천 계양 코아루 센트럴파크')
changeWord('인천 브라운스톤 계양 스카이', '브라운스톤 계양스카이')
changeWord('인천 간석 한신더휴', '간석한신더휴')
changeWord('인천간석휴먼시아(1BL)', '인천간석휴먼시아1단지')
changeWord('구월 두플라스 도시형생활주택', '두플라스')
changeWord('대명 어반스테이', '대명어반스테이')
changeWord('인천 구월 지웰시티 푸르지오', '신영구월지웰시티푸르지오')
changeWord('인천 논현 유승한내들 와이드오션', '논현유승한내들')
changeWord('인천 논현 푸르지오', '인천논현푸르지오')
changeWord('인천향촌2단지 휴먼시아', '향촌휴먼시아2단지')
changeWord('인천서창 2지구 10블록 e편한세상', '이편한세상서창')
changeWord('인천서창LH8단지아파트', '인천서창엘에이치8단지')
changeWord('인천 송림 파인앤유', '송림파인앤유 주상복합 아파트')
changeWord('인천 더샵 스카이타워 8-5블록', '더샵 인천스카이타워 2단지')
changeWord('인천 더샵 스카이타워 8-7블록', '더샵 인천스카이타워 1단지')
changeWord('인천 도화지구 금강펜테리움 센트럴파크', '도화역 금강펜테리움 센트럴파크')
changeWord('인천 SK스카이뷰 ', '인천 SK Sky VIEW')
changeWord('인천용마루 2블록 공공분양', '엘에이치미추홀퍼스트')
changeWord('인천 주안 지웰에스테이트', '주안 지웰에스테이트')
changeWord('부평 지웰 에스테이트', '부평지웰에스테이트')
changeWord('부평 코오롱하늘채', '부평코오롱하늘채')
changeWord('부평엘에이치2단지', '부평LH 2단지')
changeWord('부평 샬레 아넬리스 아파트', '부평샬레아넬리스')
changeWord('부평 아이파크', '부평IPARK')
changeWord('청천동 우민 늘푸른아파트', '청천우민늘푸른아파트')
changeWord('인천 가정지구 4블록 제일풍경채', '루원제일풍경채')
changeWord('인천 가정지구 시티 프라디움', '루원시티프라디움')
changeWord('인천가정 1블록 공공분양', '루원더퍼스트')
changeWord('인천가정 5블럭 호반베르디움 ', '루원호반베르디움더센트럴')
changeWord('인천가좌 두산위브', '인천가좌두산위브트레지움 1,2단지')
changeWord('주공2', '인천가좌두산위브트레지움 1,2단지')
changeWord('인천 검단 2차 우방아이유쉘', '검단2차우방아이유쉘')
changeWord('검단오류역 우방아이유쉘', '검단 오류역 우방아이유쉘')
changeWord('인천 청라국제도시 A1블록 모아미래도', '청라모아미래도')
changeWord('인천 청라국제도시 대광로제비앙', '청라국제 업무단지 센텀 대광로제비앙')
changeWord('청라 A12 제일풍경채2차 에듀앤파크', '청라제일풍경채2차에듀앤파크')
changeWord('청라 센트럴 에일린의 뜰 ', '청라센트럴에일린의뜰')
changeWord('청라 호수공원 한신더휴 ', '청라 호수공원 한신더휴')
changeWord('청라2차골드클래스(A23블럭)', '청라골드클래스')
changeWord('청라국제금융단지 한양수자인 레이크블루', '청라국제금융단지한양수자인레이크블루')
changeWord('청라상록힐스테이트(A10)', '청라힐스테이트커낼뷰')
changeWord('청라중흥S클래스2단지(A16)', '청라지구중흥S-CLASS2단지')
changeWord('송도 동일하이빌 파크레인', '송도 파크레인 동일하이빌')
changeWord('연수파크자이', '송도파크자이')
changeWord('인천 연수 서해그랑블 3단지', '인천연수서해그랑블3단지')
changeWord('인천 연수 행복한마을 서해그랑블', '서해그랑블 포레스트')
changeWord('인천동춘2구역연수 서해그랑블(1-2,1-3블록)', '연수 서해그랑블 1단지')
changeWord('랜드마크시티 센트럴더샵', '랜드마크시티 센트럴 더샵')
changeWord('송도 더샵 그린워크3차(D17-1블록)', '송도 더샵 그린워크 3차(17-1블럭)')
changeWord('송도 더샵 그린워크3차(D18-1블록)', '송도 더샵 그린워크 3차(18-1블럭)')
changeWord('송도 더샵 센토피아 ', '더샵송도 마리나베이')
changeWord('송도 아메리칸타운 IPARK', '송도아메리칸타운아이파크')
changeWord('송도 아트윈 푸르지오', '송도아트윈푸르지오')
changeWord('송도 호반베르디움3차 ', '호반베르디움 에듀시티')
changeWord('송도더샵마스터뷰(F21블럭)', '송도 더샵 마스터뷰 21BL')
changeWord('송도더샵퍼스트파크(F13-1블럭)', '송도 더샵 퍼스트파크 F13-1BL')
changeWord('송도더샵퍼스트파크(F14블럭)', '송도 더샵 퍼스트파크 F14BL')
changeWord('송도더샵퍼스트파크(F15블럭)', '송도 더샵 퍼스트파크 F15BL')
changeWord('송도포레스트카운티 지역주택조합', 'e편한세상 송도')
changeWord('인천 힐스테이트 레이크 송도 (A11BL)', '힐스테이트레이크송도1차')
changeWord('인천 송도 SK VIEW 아파트', '송도 SK VIEW')
changeWord('송도에듀포레푸르지오(RC-2블록)', '송도 에듀포레 푸르지오')
changeWord('송도호반베르디움2차(RC-1블럭)', '송도글로벌파크베르디움')
changeWord('힐스테이트 레이크 송도 2차', '힐스테이트레이크송도2차')
changeWord('영종하늘도시 푸르지오 자이', '영종센트럴 푸르지오자이')
changeWord('e편한세상 영종하늘도시2차', 'e편한세상 영종국제도시 오션하임')
changeWord('영종 한신더휴 스카이파크', '영종국제도시한신더휴스카이파크')
changeWord('영종하늘도시 화성파크드림', '영종국제도시 화성파크드림')
changeWord('인천영종 스카이시티자이', '스카이시티자이')

(587, 2)

(586, 2)

(585, 2)

(584, 2)

(583, 2)

...

(523, 2)

(522, 2)

(521, 2)

(520, 2)

(519, 2)

In [115]:
changeWord('화성동탄A3 상록아파트', '동탄역 센트럴상록 아파트')
changeWord('고양삼송동일스위트1차', '고양삼송동일스위트파크뷰7(1차)')
changeWord('우남퍼스트빌(A-1)', '신원마을 1단지 우남퍼스트빌')
changeWord('호반베르디움 A9블럭', 'A9블럭 신원마을 호반베르디움')
changeWord('고양 삼송 원흥역 푸르지오 (M-BL)', '삼송원흥역센트럴푸르지오')
changeWord('한강신도시 2차 KCC스위첸', '한강신도시 운양 KCC 스위첸')

(512, 2)

# 데이터 저장

In [126]:
df.to_csv('data/분양권/분양권_1619.csv', index=False, encoding='utf-8-sig')